# Production-Ready Agent Workflow with Flotorch and CrewAI

This notebook demonstrates a comprehensive, production-style workflow. It integrates all the advanced features we've explored into a single, robust application:
- **Remote Agent Configuration**: Uses `FlotorchCrewAIAgent` to load agent personas from the Flotorch UI.
- **Custom Local Tools**: Defines and attaches a custom tool to the agent at runtime.
- **Dual Memory System**: Implements both short-term session memory and long-term external memory.


### Prerequesit
Configure agent, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

## 1. Setup and Imports

The following cells install dependencies, set API credentials, and import all necessary components for handling tools, memory, and remote agent management.

In [ ]:
# install flotorch adk package
%pip install flotorch[crewai]

In [ ]:
%pip install crewai crewai_tools crewai_tools[mcp]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
MEMORY_PROVIDER = "<flotorch memory provider>"
AGENT_NAME = "<flotorch agent name>"
USER_ID = "flotorch_user_001"
APP_ID = "flotorch_app_001"

In [ ]:
# Import necessary libraries
from flotorch.crewai.agent import FlotorchCrewAIAgent
from flotorch.crewai.sessions import FlotorchCrewAISession
from flotorch.crewai.memory import FlotorchMemoryStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai.memory.external.external_memory import ExternalMemory
from crewai import Crew
from crewai.tools import tool

print("Imported necessary libraries successfully")

## 2. Custom Tool Definition

Here, we define a custom `analyze_text` tool locally. This tool can be dynamically attached to our agent at runtime, demonstrating how to extend a centrally-managed agent with new capabilities.

In [ ]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.

    Args:
        text (str): Text to analyze

    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')

    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""


print("Custom tool defined successfully.")

## 3. Memory Configuration

We configure the full dual-memory system: `ShortTermMemory` for conversational context and `ExternalMemory` for persistent, long-term knowledge.

In [ ]:
short_term_storage = FlotorchCrewAISession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

short_term_memory = ShortTermMemory(storage = short_term_storage)

external_storage = FlotorchMemoryStorage(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id  = APP_ID
)

external_memory = ExternalMemory(storage = external_storage)

print("Initialized short-term and external Memories")

## 4. Agent Initialization with Custom Tools

We initialize the `FlotorchCrewAIAgent` client, loading our agent's configuration from the Flotorch UI. Crucially, we also pass our locally defined `analyze_text` function into the `custom_tools` parameter, augmenting the agent's capabilities.

In [ ]:
flotorch_client = FlotorchCrewAIAgent(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    agent_name=AGENT_NAME,
    custom_tools = [analyze_text]
)

agent = flotorch_client.get_agent()
task = flotorch_client.get_task()

print("Fetched agent and task from Flotorch console")

## 5. Assembling the Complete Crew

Finally, we assemble the `Crew`, bringing together the remotely-configured agent, its task, the custom tool, and both memory modules. This creates a fully-featured, production-ready agent.

In [ ]:
crew = Crew(
    agents = [agent],
    tasks = [task],
    short_term_memory = short_term_memory,
    external_memory= external_memory,
    verbose = False
)

print("Created Crew with agent, task and memories")

## 6. Interactive Demonstration

Engage with the agent in this interactive loop to test its full range of capabilities. Try asking it to analyze text (to test the tool) and then ask it to recall information from the conversation. Type 'exit' to end the session.

In [ ]:
while True:

    user_query = input("user: ")

    if user_query.lower().strip() == "exit":
        break
    response = crew.kickoff(inputs = {"query":user_query})
    print(f"Assistant:{response.raw}")

## Summary

This notebook served as a **capstone project**, successfully integrating all previously demonstrated concepts into a single, comprehensive, and production-ready workflow.  
We built an advanced AI agent that combines the power of **remote configuration**, **local custom tools**, and a **full dual-memory system**.

### Key Achievements

- **Hybrid Configuration**  
  Initialized an agent using `FlotorchCrewAIAgent` to load its base configuration from the Flotorch UI, then augmented it at runtime with locally defined custom tools.

- **Full Memory Stack**  
  Equipped the agent with both `ShortTermMemory` for conversational flow and `ExternalMemory` for long-term knowledge persistence.

- **End-to-End Demonstration**  
  Showcased the agent's ability to seamlessly switch between using its custom tools for specific tasks and accessing its memory to provide contextually rich, intelligent responses.

- **Robust Architecture**  
  Delivered a complete workflow that serves as a powerful and flexible blueprint for building sophisticated, real-world AI applications.
